In [ ]:
# %%
import torch 
from PIL import Image
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader , random_split
from torchvision import transforms
print("Done")

In [ ]:
# quick sanity cell (index 1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("sdg")
print("device:", device)

In [ ]:
transforms_img = transforms.Compose([
    transforms.Resize((572, 572)),        # original U-Net input size
    transforms.ToTensor(),                # converts to [0,1]
    transforms.Normalize((0.0,), (1.0,))  # keep as-is, just scale to [0,1]
])


In [ ]:
def get_dataloaders(img_dir,
                    mask_dir,
                    batch_size: int = 64,
                    val_fraction: float = 0.2,
                    num_workers: int = 0,
                    transform=transforms_img,
                    seed: int = 42):
    """Create and return (train_loader, val_loader).

    This is the recommended way to get dataloaders from other scripts or notebooks.
    It avoids executing training code at import time and provides reproducible splits.
    """
    dataset = CarvanaDataset(img_dir=img_dir, mask_dir=mask_dir, transform=transform)
    total = len(dataset)
    n_val = int(total * val_fraction)
    n_train = total - n_val
    # use generator for reproducible split
    generator = torch.Generator().manual_seed(seed)
    train_set, val_set = random_split(dataset, [n_train, n_val], generator=generator)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_loader, val_loader


In [ ]:
# %%
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)


# %%
loss = nn.BCELoss()
model = Model().to(torch.device)
optimizer = torch.optim.Adam(model, lr=0.001)
for epoch in range(50):
    print("Epoch: ", epoch, end=" ")
    l=0
    cnt=0
    for x,y in train_loader:
        x=x.to(device)
        y=y.to(device)
        y_pred = Model(x)
        loss = loss(y_pred, y)
        l+=loss.item()
        cnt+=1
        print("Loss: ", loss.item()/x.shape[0])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print("Average Loss: ", l/(64*cnt))

# %%
with torch.no_grad():
    cnt=
    for x,y in val_loader:
        y_pred = Model(x)
        val_loss = loss(y_pred, y)
        print("Validation Loss: ", val_loss.item())